In [1]:
from owslib.wms import WebMapService
from owslib.wfs import WebFeatureService
from owslib.fes import *
from owslib.etree import etree
import json # library to parse JSON response

In [2]:
# Let's start with WMS and check out some content
wms = WebMapService("https://geo.vedur.is/geoserver/wms")
print("WMS service name: " + wms.identification.title)
print("WMS version: " + wms.identification.version)

WMS service name: Icelandic Met Office Web Map Service
WMS version: 1.1.1


In [3]:
# list all available WMS layers
list(wms.contents)

['glaciology:glacier_outline_group',
 'meteorology:IcelandicMetOffice_CAP_areas',
 'glaciology:glaciercauldronv',
 'glaciology:glacieroutline_ams',
 'glaciology:glacieroutline_ca_2000',
 'glaciology:glacieroutline_ipy',
 'glaciology:glacieroutline_lia',
 'glaciology:glacieroutline_lm_2014',
 'glaciology:glacieroutline_misc_1970s',
 'glaciology:glacieroutline_sent2_2017',
 'glaciology:glacieroutline_sent2_2019',
 'glaciology:glacierphotolocv',
 'glaciology:glacierpolygon_ams',
 'glaciology:glacierpolygon_ca_2000',
 'glaciology:glacierpolygon_ipy',
 'glaciology:glacierpolygon_lia',
 'glaciology:glacierpolygon_lm_2014',
 'glaciology:glacierpolygon_misc_1970s',
 'glaciology:glacierpolygon_sent2_2017',
 'glaciology:glacierpolygon_sent2_2019',
 'glaciology:glacierv',
 'glaciology:stakembvv',
 'glaciology:stakev',
 'tos:v_stations_wgs84']

In [4]:
# get the title for a particular layer
wms['glaciology:glacieroutline_lia'].title

'Glacier outlines - Little Ice Age, ~1890'

In [5]:
# get all available spatial reference systems for a single layer
wms['glaciology:glacieroutline_lia'].crsOptions

['EPSG:3857', 'EPSG:4326', 'EPSG:900913', 'EPSG:3057']

In [6]:
# Let's switch to the WFS service
# WFS lets you read the data directly 
wfs = WebFeatureService("https://geo.vedur.is/geoserver/wfs", version="1.1.0")
# list avaialble layers
list(wfs.contents)

['glaciology:glaciercauldronv',
 'glaciology:glacierv',
 'glaciology:stakembvv',
 'glaciology:stakev',
 'glaciology:glacieroutline_ams',
 'glaciology:glacieroutline_misc_1970s',
 'glaciology:glacieroutline_ipy',
 'glaciology:glacieroutline_lia',
 'glaciology:glacieroutline_lm_2014',
 'glaciology:glacieroutline_ca_2000',
 'glaciology:glacieroutline_sent2_2017',
 'glaciology:glacieroutline_sent2_2019',
 'glaciology:glacierphotolocv',
 'glaciology:glacierpolygon_ams',
 'glaciology:glacierpolygon_misc_1970s',
 'glaciology:glacierpolygon_ipy',
 'glaciology:glacierpolygon_lia',
 'glaciology:glacierpolygon_lm_2014',
 'glaciology:glacierpolygon_ca_2000',
 'glaciology:glacierpolygon_sent2_2017',
 'glaciology:glacierpolygon_sent2_2019',
 'glaciology:terminusobslocvv',
 'meteorology:IcelandicMetOffice_CAP_areas',
 'tos:v_stations_wgs84']

In [7]:
# get the schema (table columns names and type) for a particular layer
# this layer holds locations of monitoring stations
wfs.get_schema("tos:v_stations_wgs84")

{'properties': {'gid': 'int',
  'id_entity': 'int',
  'station_identifier': 'string',
  'marker': 'string',
  'lat': 'double',
  'lon': 'double',
  'altitude': 'decimal',
  'domain': 'string',
  'subtype': 'string',
  'operational_class': 'string',
  'owner': 'string',
  'date_start': 'dateTime',
  'date_end': 'dateTime',
  'service_bulletin': 'string'},
 'required': [],
 'geometry': 'GeometryCollection',
 'geometry_column': 'geom_wgs84'}

In [8]:
# let's create a filtered request where we ask for all features with name Hveravelli
wfs_filter = PropertyIsLike(propertyname="name", literal="Hveravellir", wildCard="*")
filterxml = etree.tostring(wfs_filter.toXML()).decode("utf-8")
response = wfs.getfeature(typename='tos:v_stations_wgs84', filter=filterxml, outputFormat='JSON')
# parse JSON response and print out the 4 features that matched our request
parsed_json = json.loads(response.read())
print(str(json.dumps(parsed_json, indent=4, sort_keys=False, ensure_ascii=False)))

{
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "id": "v_stations_wgs84.fid--57310aae_17d042d7d14_23c2",
            "geometry": {
                "type": "Point",
                "coordinates": [
                    -19.56205,
                    64.86668
                ]
            },
            "geometry_name": "geom_wgs84",
            "properties": {
                "gid": 677,
                "id_entity": 677,
                "name": "Hveravellir",
                "station_identifier": "hvrv",
                "marker": "hvrv",
                "lat": 64.86668,
                "lon": -19.56205,
                "altitude": 641,
                "domain": "meteorological",
                "subtype": "Skeytastöð",
                "operational_class": null,
                "owner": "Veðurstofa Íslands",
                "date_start": "1964-01-01T01:00:00Z",
                "date_end": "2004-01-01T00:00:00Z",
                "se